In [1]:
import time
import numpy as np
import pandas as pd
import seaborn as sb
from IPython.display import display, clear_output 
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

# set random seed to achieve same results each time
np.random.seed(23)

### Loading the Data

In [ ]:
train_csv = pd.read_csv("training.csv")
test_csv = pd.read_csv("test.csv")
sample_csv = pd.read_csv("SampleSubmission.csv")
ID_csv = pd.read_csv("IdLookupTable.csv")

### Splitting Training Data into Labels and Data

In [ ]:
def create_training(train_csv):
    label_names = train_csv.columns.tolist()[:-1]
    labels = train_csv.loc[:, train_csv.columns != 'Image'].values

    data = train_csv['Image'].str.split()
    data = np.vstack(data.apply(lambda row: np.asarray([int(n) for n in row])).values)
    data = data/255.0 # Rescale grayscale values to [0,1].
    
    # Shuffle the data
    shuffle = np.random.permutation(np.arange(data.shape[0]))
    data, labels = data[shuffle], labels[shuffle]
    
    # Splitting into dev and training
    dev_data, dev_labels = data[:1000], labels[:1000]
    train_data, train_labels = data[1000:], labels[1000:]
    
    return dev_data, dev_labels, train_data, train_labels, label_names

In [ ]:
dev_data, dev_labels, train_data, train_labels, label_names = create_training(train_csv)

### Plotting an Example

In [ ]:
def plot_example(image_number):
    plt.imshow(dev_data[image_number].reshape(96, 96), cmap='gray')
    plt.axis('off')
    plt.scatter(dev_labels[image_number][0], dev_labels[image_number][1], c='red', marker='o')
    
    plt.annotate('left eye center', xy         = (dev_labels[image_number][0], 
                                                  dev_labels[image_number][1]), 
                                    xytext     = (dev_labels[image_number][0]+5, 
                                                  dev_labels[image_number][1]+5),
                                    arrowprops = {'facecolor': 'black', 'shrink': 0.05, 
                                                  'width':1.2, 'headwidth': 5})
    return

plot_example(5)

### Simple Classifers

We note that there are a ton of missing labels. Below is the percentage of labels for each Facial Keypoint in the training data set

In [ ]:
(train_csv.loc[:, train_csv.columns != 'Image'].count()/len(train_csv.index)).round(3)

We will attempt to fit simple classifers to the labels of the Facial Keypoints with the highest amount of non-missing labels: 
- left_eye_center 
- right_eye_center 
- nose_tip 
- mouth_center_bottom_lip

#### Left Eye Center

Let's examine some summary statistics of this feature

In [ ]:
left_eye_center_x, left_eye_center_y = train_labels[:, 0], train_labels[:, 1]

In [ ]:
plt.hist(left_eye_center_x[np.isfinite(left_eye_center_x)], bins=np.arange(35, 95, 0.5), label = 'x coordinate');
plt.hist(left_eye_center_y[np.isfinite(left_eye_center_y)], bins=np.arange(0, 85, 0.5),  label = 'y coordinate',
         color='green');
plt.title('Left Eye Center Coordinates', fontsize=12)
plt.xlabel('X & Y Coordinate Values',    fontsize=12)
plt.legend()

#### x-cordinate of left eye

Since our coordinate values of our labels are continuous, we will use K-NN Regression

In [ ]:
k_nn_xcoord = KNeighborsRegressor(radius=1)

In [ ]:
k_nn_xcoord.fit(train_data[np.isfinite(train_labels[:, 0])], 
                train_labels[:, 0][np.isfinite(train_labels[:, 0])])

In [ ]:
x_pred = k_nn_xcoord.predict(dev_data[np.isfinite(dev_labels[:, 0])])

In [ ]:
np.sqrt(mean_squared_error(dev_labels[:, 0][np.isfinite(dev_labels[:, 0])], 
                           x_pred))

#### y-cordinate of left eye

In [ ]:
k_nn_ycoord = KNeighborsRegressor(radius=1)

In [ ]:
k_nn_ycoord.fit(train_data[np.isfinite(train_labels[:, 1])], 
         train_labels[:, 1][np.isfinite(train_labels[:, 1])])

In [ ]:
y_pred = k_nn_ycoord.predict(dev_data[np.isfinite(dev_labels[:, 1])])

In [ ]:
np.sqrt(mean_squared_error(dev_labels[:, 1][np.isfinite(dev_labels[:, 1])], 
                           y_pred))

#### Visualizing Result

In [ ]:
x = k_nn_xcoord.predict(dev_data[0].reshape(1, -1))
y = k_nn_ycoord.predict(dev_data[0].reshape(1, -1))

In [ ]:
plt.figure()

for n in range(1, 3):
    plt.subplot(1, 3, n)
    plt.imshow(dev_data[n].reshape(96, 96), cmap='gray')
    plt.scatter(dev_labels[n,0], dev_labels[n,1], color='red', label='actual left eye center')
    plt.scatter(x, y, color='blue', label='predicted left eye center')
# plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
#            fancybox=True,      shadow=True, ncol=2, fontsize=12)

# plt.subplot(1, 3, 2)
# plt.imshow(dev_data[1].reshape(96, 96), cmap='gray')
# plt.scatter(dev_labels[1,0], dev_labels[1,1], color='red', label='actual left eye center')
# plt.scatter(x, y, color='blue', label='predicted left eye center')
# # plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
# #            fancybox=True,      shadow=True, ncol=2, fontsize=12)

# plt.subplot(1, 3, 3)
# plt.imshow(dev_data[10].reshape(96, 96), cmap='gray')
# plt.scatter(dev_labels[10,0], dev_labels[10,1], color='red', label='actual left eye center')
# plt.scatter(x, y, color='blue', label='predicted left eye center')
# # plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
# #            fancybox=True,      shadow=True, ncol=2, fontsize=12)

plt.legend(bbox_to_anchor=(1.05, 0), loc='lower left', borderaxespad=0.)